# First data imports and visualization

## imports

In [ ]:
import pandas as pd
import geopandas as gpd
import numpy as np
import pypsa
import datetime
import seaborn as sns
import cartopy
import cartopy.crs as ccrs
import matplotlib.colors as mcolors
import matplotlib.pyplot as plt

from utils import market_values, market_values_by_time_index, nodal_balance, carrier_colors

In [ ]:
n =pypsa.Network("../data/raw/elec_s_181_lv1.0__Co2L0-3H-T-H-B-I-A-solar+p3-linemaxext10-noH2network_2030.nc")

In [ ]:
# n.export_to_csv_folder("../data/processed/elec_s_181_lv1.0__Co2L0-3H-T-H-B-I-A-solar+p3-linemaxext10-noH2network_2030.nc")

## key data
- 365 * 24 / 3 = 2920 snapshots over one year (every 3 hours) -> rows
- 3086 different buses
- 181 different locations (+ EU: 182)
- 1707 different generators
- 8375 links
- 1635 stores
- 170 storage units


In [ ]:
# number of buses
n.storage_units

In [ ]:
# Energy carrier, such as AC, DC, heat, wind, PV or coal. Buses have direct carriers and Generators indicate their primary energy carriers. The Carrier can track properties relevant for global constraints, such as CO2 emissions.
n.carriers

In [ ]:
# counts of generators by carrier
n.generators.carrier.value_counts()

In [ ]:
# show the generators (electricity feed in) over time
n.generators_t.p
# 1707 different generators

In [ ]:
# 3086 nodes / buses: Electrically fundamental node where x-port objects attach.
n.buses

In [ ]:
# PQ power consumer.
n.loads_t.p

In [ ]:
# Storage unit with fixed nominal-energy-to-nominal-power ratio.
n.storage_units

## network visualization

In [ ]:
m = n.copy()
m.mremove("Bus",m.buses[n.buses.x == 0].index )

In [ ]:
fig, ax = plt.subplots(1, 1, subplot_kw={"projection": ccrs.EqualEarth()}, figsize=(8, 8))

m.plot(ax=ax, projection=ccrs.EqualEarth())
plt.show()

In [ ]:
fig, ax = plt.subplots(1, 1, subplot_kw={"projection": ccrs.EqualEarth()}, figsize=(8, 8))

bus_locs = pd.Index(m.buses.location.unique())
load_distribution = m.loads_t.p[bus_locs].sum()/m.loads_t.p[bus_locs].sum().max()

m.plot(bus_sizes=1*load_distribution, ax=ax, title="Load distribution", projection=ccrs.EqualEarth())
plt.show()

In [ ]:
# line loading plot
snap = n.snapshots[0]
loading = m.lines_t.p0.loc[snap] / m.lines.s_nom
loading[loading.isnull()] = 0

fig, ax = plt.subplots(subplot_kw={"projection": ccrs.EqualEarth()}, figsize=(9, 9))
cmap= plt.cm.OrRd
norm = mcolors.Normalize(vmin=0, vmax=1)
sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
sm.set_array([])

colors = list(map(mcolors.to_hex, cmap(norm(loading))))

m.plot(
    ax=ax,
    line_colors=colors,
    line_cmap=plt.cm.jet,
    title="Line loading",
    bus_sizes=1e-3,
    bus_alpha=0.7,
)

plt.colorbar(sm, orientation='vertical', shrink=0.7, ax=ax)
fig.tight_layout()

In [ ]:
fig, ax = plt.subplots(subplot_kw={"projection": ccrs.PlateCarree()}, figsize=(8, 8))

plt.hexbin(
    m.buses.x,
    m.buses.y,
    gridsize=20,
    C=m.buses_t.marginal_price.loc[m.snapshots[0]],
    cmap=plt.cm.jet,
    zorder=3,
)
m.plot(ax=ax, line_widths=pd.Series(0.5, m.lines.index), bus_sizes=0)

cb = plt.colorbar(location="bottom")
cb.set_label("Locational Marginal Price (EUR/MWh)")
fig.tight_layout()

## Generation & Consumption

#### AC

In [ ]:
carrier = ["AC"]
nb_ac = nodal_balance(n, carrier = carrier, time="2013", aggregate=['component', 'bus'], energy=True)  # in units of energy
# convert from MW to GW
nb_ac = nb_ac.unstack(level=[1]) / 1000
nb_ac.head()

In [ ]:
nb_ac_pos = nb_ac.sum()[nb_ac.sum() > 0].sort_values(ascending=False)
# exclude all shares smaller than 0.1 %
nb_ac_pos = nb_ac_pos[ (nb_ac_pos/ nb_ac_pos.sum()) > 0.005]

In [ ]:
fig, ax = plt.subplots(figsize=(6, 7))

c = [carrier_colors[col] for col in nb_ac_pos.index]
percents = nb_ac_pos.to_numpy() * 100 / nb_ac_pos.to_numpy().sum()
labels = ['%s (%1.1f %%)' % (l, s) for l, s in zip(nb_ac_pos.index,percents)]

patches, texts = ax.pie(nb_ac_pos, colors=c, startangle=0, labels=labels)
ax.axis('equal')

plt.title(f"Generation: {carrier}")
plt.show()

In [ ]:
nb_ac_neg = abs(nb_ac.sum()[nb_ac.sum() < 0]).sort_values(ascending=False)
# exclude all shares smaller than 0.1 %
nb_ac_neg = nb_ac_neg[ (nb_ac_neg/ nb_ac_neg.sum()) > 0.005]

In [ ]:
fig, ax = plt.subplots(figsize=(6, 7))

c = [carrier_colors[col] for col in nb_ac_neg.index]
percents = nb_ac_neg.to_numpy() * 100 / nb_ac_neg.to_numpy().sum()
labels = ['%s (%1.1f %%)' % (l, s) for l, s in zip(nb_ac_neg.index,percents)]

patches, texts = ax.pie(nb_ac_neg, colors=c, startangle=0, labels=labels)
ax.axis('equal')

plt.title(f"Consumption: {carrier}")
plt.show()

#### H2

In [ ]:
carrier = ["H2"]
nb_h2 = nodal_balance(n, carrier = carrier, time="2013", aggregate=['component', 'bus'], energy=True)  # in units of energy
# convert from MW to GW
nb_h2 = nb_h2.unstack(level=[1]) / 1000
nb_h2.head()

In [ ]:
# what about the fuel cell?
plt.plot(nb_h2["H2 Fuel Cell"])

In [ ]:
nb_h2_pos = nb_h2.sum()[nb_h2.sum() > 0].sort_values(ascending=False)
# exclude all shares smaller than 0.1 %
nb_h2_pos = nb_h2_pos[ (nb_h2_pos/ nb_h2_pos.sum()) > 0.005]

In [ ]:
fig, ax = plt.subplots(figsize=(6, 7))

c = [carrier_colors[col] for col in nb_h2_pos.index]
percents = nb_h2_pos.to_numpy() * 100 / nb_h2_pos.to_numpy().sum()
labels = ['%s (%1.1f %%)' % (l, s) for l, s in zip(nb_h2_pos.index,percents)]

patches, texts = ax.pie(nb_h2_pos, colors=c, startangle=0, labels=labels)
ax.axis('equal')

plt.title(f"Generation: {carrier}")
plt.show()

In [ ]:
# Fuel Cell & H2 are irrelevant
abs(nb_h2.sum()[nb_h2.sum() < 0]).sort_values(ascending=False)

In [ ]:
nb_h2_neg = abs(nb_h2.sum()[nb_h2.sum() < 0]).sort_values(ascending=False)
# exclude all shares smaller than 0.1 %
nb_h2_neg = nb_h2_neg[ (nb_h2_neg/ nb_h2_neg.sum()) > 0.005]

In [ ]:
fig, ax = plt.subplots(figsize=(6, 7))

c = [carrier_colors[col] for col in nb_h2_neg.index]
percents = nb_h2_neg.to_numpy() * 100 / nb_h2_neg.to_numpy().sum()
labels = ['%s (%1.1f %%)' % (l, s) for l, s in zip(nb_h2_neg.index,percents)]

patches, texts = ax.pie(nb_h2_neg, colors=c, startangle=0, labels=labels)
ax.axis('equal')

plt.title(f"Consumption: {carrier}")
plt.show()

#### Electricity

In [ ]:
carrier = ["AC", "battery", "Li ion", "low voltage", "home battery"]
nb_el = nodal_balance(n, carrier = carrier, time="2013", aggregate=['component', 'bus'], energy=True)  # in units of energy
# convert from MW to GW
nb_el = nb_el.unstack(level=[1]) / 1000
nb_el.head()

In [ ]:
# Generation
nb_el_pos = nb_el.sum()[nb_el.sum() > 0].sort_values(ascending=False)
# exclude all shares smaller than 0.5 %
nb_el_pos = nb_el_pos[(nb_el_pos / nb_el_pos.sum()) > 0.005]

fig, ax = plt.subplots(figsize=(6, 7))

c = [carrier_colors[col] for col in nb_el_pos.index]
percents = nb_el_pos.to_numpy() * 100 / nb_el_pos.to_numpy().sum()
labels = ['%s (%1.1f %%)' % (l, s) for l, s in zip(nb_el_pos.index, percents)]

patches, texts = ax.pie(nb_el_pos, colors=c, startangle=0, labels=labels)
ax.axis('equal')

plt.title(f"Generation: {carrier}")
plt.show()

In [ ]:
# Consumption
nb_el_neg = abs(nb_el.sum()[nb_el.sum() < 0]).sort_values(ascending=False)
# exclude all shares smaller than 0.5 %
nb_el_neg = nb_el_neg[(nb_el_neg / nb_el_neg.sum()) > 0.005]

fig, ax = plt.subplots(figsize=(6, 7))

c = [carrier_colors[col] for col in nb_el_neg.index]
percents = nb_el_neg.to_numpy() * 100 / nb_el_neg.to_numpy().sum()
labels = ['%s (%1.1f %%)' % (l, s) for l, s in zip(nb_el_neg.index, percents)]

patches, texts = ax.pie(nb_el_neg, colors=c, startangle=0, labels=labels)
ax.axis('equal')

plt.title(f"Consumption: {carrier}")
plt.show()

## first plots

In [ ]:
# create some interesting indices

# first hour: 1.1.2013 00:00:00 - 01:00:00
hour1 = n.generators_t.p.index <= datetime.datetime(2013,1,1,1)
# first day: 1.1.2013
day1 = n.generators_t.p.index <= datetime.datetime(2013,1,1)
# first month: january
month1 = n.generators_t.p.index < datetime.datetime(2013,1,31)

In [ ]:
# generation of generator 'AL0 0 offwind-ac' for first month
plt.plot(n.generators_t.p['AL0 0 offwind-ac'][month1])

In [ ]:
# names of the different generators
n.generators_t.p.columns

In [ ]:
plt.plot(n.generators_t.p[month1]['AL0 0 solar'])

In [ ]:
# market values: "onwind", "solar", "solar rooftop", "offwind-dc", "offwind-ac"
mv_onwind = market_values(n, "onwind")
mv_onwind

In [ ]:
plt.plot(mv_onwind,'x')

In [ ]:
# shadow prices (in €/MWh) for every nod/bus and time step
# (e.g. Locational Marginal Price (LMP) for electricity)
n.buses_t.marginal_price

In [ ]:
carrier = "onwind"
# select all the generators for specific carrier e.g. onwind
gen = n.generators_t.p.loc[:, n.generators.carrier == carrier]
# create index from the generators and use as column names
gen.columns = gen.columns.map(n.generators.bus)
# get locational marginal prices for all locations and all time steps
lmp = n.buses_t.marginal_price.loc[:, gen.columns]

In [ ]:
# plot of lmp of AL0 0
plt.plot(lmp["AL0 0"])

In [ ]:
# calculate market values as sum product of generation times lmp divided by total generation
mv = (gen * lmp).sum() / gen.sum()
# set location of the buses/nodes as the index -> shape = (n_generators x 1) 1 col with mvs
mv.index = mv.index.map(n.buses.location)

In [ ]:
# plot: carrier market value plotted against carrier share
market_share = gen.sum(axis=1) / n.generators_t.p.sum(axis=1)
plt.scatter(market_share, lmp.mean(axis=1))

In [ ]:
plt.plot(market_share[month1])
plt.plot((lmp.mean(axis=1)/lmp.max(axis=1))[month1])

In [ ]:
lmp.mean(axis=1)

In [ ]:
market_share

In [ ]:
test = pd.concat([market_share, lmp.mean(axis=1)], axis=1)
test.columns = ["market_share", "lmp"]
test.set_index("market_share", inplace=True)
test.sort_index(ascending=True)

In [ ]:
plt.scatter(test.index, test.lmp)
m, b = np.polyfit(test.index, test.lmp, 1)
plt.plot(test.index, m*test.index+b, color="red")

In [ ]:
mv_onwind = market_values(n,"onwind")

In [ ]:
# Link between two buses with controllable active power - can be used for a transport power flow model OR as a simplified version of point-to-point DC connection OR as a lossy energy converter. NB: for a lossless bi-directional HVDC or transport link, set p_min_pu = -1 and efficiency = 1. NB: It is assumed that the links neither produce nor consume reactive power.

n.links_t.p0

In [ ]:
n.components

In [ ]:
# market value for the 181 onwind generators (k-mean clusters)
mv_onwind

In [ ]:
p_by_carrier = n.generators_t.p.groupby(n.generators.carrier, axis=1).sum()
p_by_carrier.drop(
    (p_by_carrier.max()[p_by_carrier.max() < 1700.0]).index, axis=1, inplace=True
)
p_by_carrier.columns

In [ ]:
colors = {
    "oil": "k",
    "offwind-dc": "r",
    "onwind": "green",
    "ror": "blue",
    "solar": "yellow",
    "offwind-ac": "cyan",
    "solar rooftop": "brown",
    "gas": "orange",
    "urban central solar thermal": "pink"
}
# reorder
cols = [
    "oil",
    "gas",
    "ror",
    "offwind-ac",
    "offwind-dc",
    "onwind",
    "solar",
    "solar rooftop",
    "urban central solar thermal",
]
p_by_carrier = p_by_carrier[cols]

In [ ]:
p_by_carrier.max()

In [ ]:
(p_by_carrier / 1e3).max()

In [ ]:
index = (p_by_carrier.index > datetime.datetime(2013,3,31)) & (p_by_carrier.index < datetime.datetime(2013,5,1)) # huge peak for 16.02
c = [colors[col] for col in p_by_carrier.columns]

fig, ax = plt.subplots(figsize=(12, 6))
(p_by_carrier[index] / 1e3).plot(kind="area", ax=ax, linewidth=4, color=c, alpha=0.7)
ax.legend(ncol=4, loc="upper left")
ax.set_ylabel("GW")
ax.set_xlabel("")
fig.tight_layout()

In [ ]:
p_by_carrier[index]

In [ ]:
fig, ax = plt.subplots(figsize=(12, 6))

p_storage = n.storage_units_t.p.sum(axis=1)
state_of_charge = n.storage_units_t.state_of_charge.sum(axis=1) / 1e3
p_storage.plot(label="Pumped hydro dispatch (MWh)", ax=ax, linewidth=3)
state_of_charge.plot(label="State of charge (GWh)", ax=ax, linewidth=3)

ax.legend()
ax.grid()
ax.set_ylabel("")
ax.set_xlabel("")
fig.tight_layout()

In [ ]:
# marginal prices
n.buses_t.marginal_price.loc[snap].describe()

In [ ]:
n.buses[n.buses.x != 0].index

In [ ]:
n.buses_t.marginal_price.loc[snap,n.buses[n.buses.x != 0].index]

In [ ]:
fig, ax = plt.subplots(subplot_kw={"projection": ccrs.PlateCarree()}, figsize=(8, 8))

plt.hexbin(
    n.buses[n.buses.x != 0].x,
    n.buses[n.buses.x != 0].y,
    gridsize=20,
    C=n.buses_t.marginal_price.loc[snap,n.buses[n.buses.x != 0].index],
    cmap=plt.cm.jet,
    zorder=3,
)
# n.plot(ax=ax, line_widths=pd.Series(0.5, n.lines.index), bus_sizes=0)

cb = plt.colorbar(location="bottom")
cb.set_label("Locational Marginal Price (EUR/MWh)")
fig.tight_layout()

## Market values general investigation

In [ ]:
carriers_buses = n.buses.carrier.unique().tolist()
carriers_gens = n.generators.carrier.unique().tolist()
carriers_links = n.links.carrier.unique().tolist()
carriers_buses
carriers_gens
carriers_links

In [ ]:
# counts of number of regions where a generator of e specific carrier is present
n.generators.carrier.value_counts()

In [ ]:
# market values for generators
mv_carriers = pd.DataFrame(index = n.buses.location.unique(), columns = carriers_gens)

for carrier in carriers_gens:
    mv_carriers[carrier] = market_values(n, carrier)

mv_carriers

In [ ]:
# plot overall market value for all carriers in generators

mv_carriers_mean = pd.DataFrame(mv_carriers.mean())
mv_carriers_mean.columns = ["market_values"]
sns.set(rc={'figure.figsize':(8,6)})
sns.barplot(data=mv_carriers_mean,y=mv_carriers_mean.index, x="market_values", orient="h")
ticks = plt.xticks(rotation=90)

In [ ]:
# Plot of mv of carriers by country
mv_carriers["country"] = mv_carriers.index.str[:2]
ax = mv_carriers.groupby(by="country").mean().plot(figsize=(20,10))
plt.legend(loc='upper left', ncol=3, prop={'size': 12})
ax.set_xticks(range(len(mv_carriers.groupby(by="country").mean())))
ticks = ax.set_xticklabels(labels = [item for item in mv_carriers.groupby(by="country").mean().index.tolist()], rotation=0)

In [ ]:
# # Plot of market values of carriers for periods
days = pd.DatetimeIndex(np.unique(n.generators_t.p.index.date))
# plt.figure(figsize=(15, 10))
#
# for carrier in car_gen_vre:
#     mv = market_values_by_time_index(n, days, carrier)
#     plt.plot(mv.mean(axis=1), label=carrier)
#
# plt.legend()

In [ ]:
# # Plot of all locational marginal prices over all carriers per regions
lmp_regions = n.buses_t.marginal_price.loc[:, n.buses.location.unique()[:-1]]
# df1 = lmp_regions.mean()
# sns.set(rc={'figure.figsize':(10,30)})
# ax = sns.barplot(y=df1.index, x=df1.values)
#
# for i in ax.containers:
#     ax.bar_label(i,)

In [ ]:
# # Plot of lmp over all carriers averaged over countries
# df2 = lmp_regions.transpose()
# df2["country"] = df2.index.str[:2]
# df2 = df2.groupby(by="country").mean().mean(axis=1)
# sns.set(rc={'figure.figsize':(10,7)})
# ax = sns.barplot(y=df2.index, x=df2.values)
#
# for i in ax.containers:
#     ax.bar_label(i,)

In [ ]:
# Plot of lmp daily pattern of lmps over all carriers normalized by the mean price of the day
plt.figure(figsize=(12, 8))

carrier_bus = "H2" #carriers_buses
locs = n.buses.location[n.buses[n.buses.carrier == carrier_bus].index]
lmps = n.buses_t.marginal_price[n.buses[n.buses.carrier == carrier_bus].index]

for day in days:
    df = lmps[lmps.index.date == day.date()].mean(axis=1)
    df_normalized = df / df.mean()
    plt.plot(df_normalized.index.hour, df_normalized.values)

In [ ]:
# # Plot of lmp daily pattern of lmps over all carriers normalized by the mean price of the current day
plt.figure(figsize=(12, 8))
df3 = pd.DataFrame(lmps.mean(axis=1))
df3.columns = ["lmp"]
df3["lmp_normalized"] = np.nan

for snap in df3.index:
    df3.loc[snap, "lmp_normalized"] = df3.loc[snap, "lmp"] / df3[df3.index.date == snap.date()].lmp.mean()

df3["hour_of_day"] = df3.index.hour
sns.boxplot(data=df3, y="lmp_normalized", x="hour_of_day")
plt.show()

In [ ]:
# simulation fractal
from shapely.geometry import Point
import random

def midpoint(p1, p2):
    return Point((p1.x+p2.x)/2, (p1.y+p2.y)/2)

def get_coords(list):
    xs = [point.x for point in list]
    ys = [point.y for point in list]
    return xs, ys

n = 11*365
a, b, c = Point(0,2), Point(0,3), Point(2, 0)
start_point = Point(1, 2)
points = [a, b, c]
more_points = []

fig = plt.figure(figsize=(10, 15))
xs, ys = get_coords(points)
plt.scatter(xs, ys, marker="h", color="darkorange")

plt.scatter(start_point.x, start_point.y, marker="*", color="darkorange")

for i in range(n):
    next_point = midpoint(start_point, random.choice(points))
    more_points.append(next_point)
    start_point = next_point

xs, ys = get_coords(more_points)
plt.scatter(xs, ys, s=0.9, color="darkviolet")

plt.grid(False)
plt.axis('off')
fig.set_facecolor('azure')

plt.show()